In [1]:
# author: Yuhan Jia (jiayuhan21@mails.ucas.ac.cn) and Yiyang Zhang (zhangyiyang1328@163.com)
import sys
sys.path.append('./Algorithm/STAligner/STAligner/')

import ST_utils
import train_STAligner

###################################
import os
os.environ['R_HOME'] = "D:/Users/zyzhang/R-4.2.1"
os.environ['R_USER'] = "D:/Users/zyzhang/Miniconda/envs/mouse_AD/Lib/site-packages/rpy2"

import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri

import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import KMeans
import itertools
import torch
import math
# from evaluation import eva
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import os
import scipy.sparse as sp
import scipy.linalg
from sklearn.metrics import adjusted_rand_score as ari_score
import torch.nn.functional as F

# seed_everything()
import torch
import numpy as np
seed=666
import torch.backends.cudnn as cudnn
cudnn.deterministic = True
cudnn.benchmark = True
import random
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

In [2]:
Batch_list = []

section_ids = ['ST_3M_AD_1_1', 'ST_3M_AD_1_2', 'ST_3M_WT_1_1', 'ST_3M_WT_1_2',
               'ST_6M_AD_2_1', 'ST_6M_AD_2_2', 'ST_6M_WT_2_1', 'ST_6M_WT_2_2',
               'ST_15M_AD_2_1', 'ST_15M_AD_2_2', 'ST_15M_WT_2_1', 'ST_15M_WT_2_2'] 

In [3]:
for section_id in section_ids:
    print(section_id)
    
    # The path to Data
    input_dir0 = os.path.join('D:/Users/zyzhang/Alzheimer/st/', section_id)
    adata = sc.read_visium(path=input_dir0, count_file='filtered_feature_bc_matrix.h5',
                           load_images=True)
    
    if_choose_file = pd.read_csv(input_dir0 + '/if_choose.csv', index_col=0, sep=',')
    barcodes_for_choose = if_choose_file[if_choose_file['if_choose']== 1].index 
    adata = adata[barcodes_for_choose, ]
    
    adata.var_names_make_unique(join="++")
    
    # Normalization
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=15000)

    Batch_list.append(adata)

ST_3M_AD_1_1
ST_3M_AD_1_2
ST_3M_WT_1_1
ST_3M_WT_1_2
ST_6M_AD_2_1
ST_6M_AD_2_2
ST_6M_WT_2_1
ST_6M_WT_2_2
ST_15M_AD_2_1
ST_15M_AD_2_2
ST_15M_WT_2_1
ST_15M_WT_2_2


In [4]:
for m in range(len(Batch_list)):
    # Read the clustering results from STAligner
    STAligner_results = pd.read_csv(f'./results/STAligner/256hidden/{section_ids[m]}.csv',
                                    index_col=0, sep=',', header=0)
    
    Batch_list[m].obs['STAligner'] = STAligner_results.loc[Batch_list[m].obs_names, 
                                                           'STAligner'].map(lambda x: str(x))

In [5]:
map_dict = {
    "1": "L5",
    "2": "AIp",
    "3": "L6",
    "4": "CTXsp-1",
    "5": "L4",
    "6": "TH-2",
    "7": "SI",
    "8": "HY",
    "9": "CA-1",
    "10": "sAMY",
    "11": "L2/3",
    "12": "L1",
    "13": "STRD-1",
    "14": "OPT",
    "15": "PIR-1",
    "16": "TH-1",
    "17": "CA-2",
    "18": "VL",
    "19": "DORsm",
    "20": "CC",
    "21": "STRD-2",
    "22": "PIR-2",
    "23": "DG",
    "24": "CTXsp-2",
    "25": "RT"
}

In [6]:
import tqdm

if not os.path.exists(f'./figures/show/final_STAligner'):
    os.makedirs(f'./figures/show/final_STAligner')

for m in tqdm.tqdm(range(len(Batch_list))):
    
    Batch_list[m].obs['STAligner'] = Batch_list[m].obs['STAligner'].map(lambda x:map_dict[x])
    
    # sc.pl.embedding(Batch_list[m], color=[f'STAligner'], basis='spatial',
    #                   frameon = False, size=80, title='', show=False,
    #                   save=f'/final_STAligner/{section_ids[m]}.png')
    # sc.pl.embedding(Batch_list[m], color=[f'STAligner'], basis='spatial',
    #                   frameon = False, size=80, title='', show=False,
    #                   save=f'/final_STAligner/{section_ids[m]}.pdf')
    sc.pl.spatial(
            Batch_list[m],
            color=[f'STAligner'],
            size=1.3,
            img_key="hires",
        title='', frameon=False,
            show=False,
            save=f'/final_STAligner/{section_ids[m]}.png',
        )
    sc.pl.spatial(
            Batch_list[m],
            color=[f'STAligner'],
            size=1.3,
            img_key="hires",
        title='', frameon=False,
            show=False,
            save=f'/final_STAligner/{section_ids[m]}.pdf',
        )

  0%|                                                                                                                                                                                      | 0/12 [00:00<?, ?it/s]

  8%|██████████████▌                                                                                                                                                               | 1/12 [00:04<00:51,  4.67s/it]

 17%|█████████████████████████████                                                                                                                                                 | 2/12 [00:08<00:38,  3.89s/it]

 25%|███████████████████████████████████████████▌                                                                                                                                  | 3/12 [00:11<00:33,  3.67s/it]

 33%|██████████████████████████████████████████████████████████                                                                                                                    | 4/12 [00:14<00:28,  3.58s/it]

...

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 8/12 [00:28<00:13,  3.43s/it]

 75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                           | 9/12 [00:31<00:10,  3.46s/it]

 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 10/12 [00:35<00:06,  3.39s/it]

 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌              | 11/12 [00:38<00:03,  3.41s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:42<00:00,  3.50s/it]


In [7]:
Merge_regions_dict = {
    'L2/3': 'Cerebral cortex',
    'CTXsp-1': 'Cerebral nuclei',
    'DG': 'Hippocampal region',
    'CA-2': 'Hippocampal region',
    'sAMY': 'Cerebral nuclei',
    'AIp': 'Cerebral cortex',
    'HY': 'Hypothalamus',
    'PIR-1': 'Cerebral cortex',
    'PIR-2': 'Cerebral cortex',
    'L5': 'Cerebral cortex',
    'CC': 'Fiber tracts',
    'SI': 'Cerebral nuclei',
    'CA-1': 'Hippocampal region',
    'L6': 'Cerebral cortex',
    'L1': 'Cerebral cortex',
    'VL': 'Ventricular systems',
    'DORsm': 'Thalamus',
    'TH-2': 'Thalamus',
    'CTXsp-2': 'Cerebral nuclei',
    'TH-1': 'Thalamus',
    'OPT': 'Fiber tracts',
    'STRD-1': 'Cerebral nuclei',
    'L4': 'Cerebral cortex',
    'STRD-2': 'Cerebral nuclei',
    'RT': 'Thalamus'
}

In [8]:
import tqdm

if not os.path.exists(f'./figures/show/Merge_regions'):
    os.makedirs(f'./figures/show/Merge_regions')

for m in tqdm.tqdm(range(len(Batch_list))):
    Batch_list[m].obs['Merge_regions'] = Batch_list[m].obs['STAligner'].map(lambda x:Merge_regions_dict[x])
    
    # sc.pl.embedding(Batch_list[m], color=[f'Merge_regions'], basis='spatial',
    #                   frameon = False, size=80, title='', show=False,
    #                   save=f'/Merge_regions/{section_ids[m]}.png')
    # sc.pl.embedding(Batch_list[m], color=[f'Merge_regions'], basis='spatial',
    #                   frameon = False, size=80, title='', show=False,
    #                   save=f'/Merge_regions/{section_ids[m]}.pdf')
    sc.pl.spatial(
            Batch_list[m],
            color=[f'Merge_regions'],
            size=1.3,
            img_key="hires",
            show=False,
        frameon=False, title='',
            save=f'/Merge_regions/{section_ids[m]}.png',
        )
    sc.pl.spatial(
            Batch_list[m],
            color=[f'Merge_regions'],
            size=1.3,
            img_key="hires",
            show=False,
        frameon=False, title='',
            save=f'/Merge_regions/{section_ids[m]}.pdf',
        )

  0%|                                                                                                                                                                                      | 0/12 [00:00<?, ?it/s]

  8%|██████████████▌                                                                                                                                                               | 1/12 [00:02<00:25,  2.34s/it]

 17%|█████████████████████████████                                                                                                                                                 | 2/12 [00:05<00:26,  2.69s/it]

 25%|███████████████████████████████████████████▌                                                                                                                                  | 3/12 [00:07<00:23,  2.57s/it]

 33%|██████████████████████████████████████████████████████████                                                                                                                    | 4/12 [00:10<00:20,  2.61s/it]

...

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 8/12 [00:20<00:10,  2.60s/it]

 75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                           | 9/12 [00:23<00:07,  2.66s/it]

 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 10/12 [00:26<00:05,  2.69s/it]

 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌              | 11/12 [00:28<00:02,  2.62s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:31<00:00,  2.63s/it]


In [10]:
for m in tqdm.tqdm(range(len(Batch_list))):
    Batch_list[m].uns['Merge_regions_colors'] = ['#6cad97', '#a8d3ea', '#c6c9cc', '#93ce91', '#dd6d6a', 
                                                 '#ea90a0', '#a2a3b7']
    
    # sc.pl.embedding(Batch_list[m], color=[f'Merge_regions'], basis='spatial',
    #                   frameon = False, size=80, title='', show=False,
    #                   save=f'/Merge_regions/{section_ids[m]}.png')
    # sc.pl.embedding(Batch_list[m], color=[f'Merge_regions'], basis='spatial',
    #                   frameon = False, size=80, title='', show=False,
    #                   save=f'/Merge_regions/{section_ids[m]}.pdf')
    sc.pl.spatial(
            Batch_list[m],
            color=[f'Merge_regions'],
            size=1.3,
            img_key="hires",
            show=False, frameon=False, title='',
            save=f'/Merge_regions/{section_ids[m]}.png',
        )
    sc.pl.spatial(
            Batch_list[m],
            color=[f'Merge_regions'],
            size=1.3,
            img_key="hires",
            show=False, frameon=False, title='',
            save=f'/Merge_regions/{section_ids[m]}.pdf',
        )

  0%|                                                                                                                                                                                      | 0/12 [00:00<?, ?it/s]

  8%|██████████████▌                                                                                                                                                               | 1/12 [00:02<00:30,  2.74s/it]

 17%|█████████████████████████████                                                                                                                                                 | 2/12 [00:05<00:26,  2.64s/it]

 25%|███████████████████████████████████████████▌                                                                                                                                  | 3/12 [00:07<00:23,  2.63s/it]

 33%|██████████████████████████████████████████████████████████                                                                                                                    | 4/12 [00:10<00:20,  2.57s/it]

...

 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 8/12 [00:20<00:10,  2.61s/it]

 75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                           | 9/12 [00:23<00:07,  2.64s/it]

 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 10/12 [00:26<00:05,  2.68s/it]

 92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌              | 11/12 [00:28<00:02,  2.61s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:31<00:00,  2.62s/it]
